In [ ]:
import pandas as pd
import os

import sys
sys.path.append("..")
from src.Eval import aggregated_results_llm, results_to_dataframe, silent_eval_llm, get_iobs_from_data, parse_filename
from src.Utils import parse_pred_file, parse_preds, read_jsonlines, read_IOB_file

results_postcut = aggregated_results_llm("../output/reddit+shsyt_cloze/post_cutoff")
results_seen = aggregated_results_llm("../output/reddit+shsyt_cloze/seen")
results_unseen = aggregated_results_llm("../output/reddit+shsyt_cloze/unseen")


In [ ]:
results_to_dataframe(results_seen)

In [ ]:
results_to_dataframe(results_unseen)

In [ ]:
results_to_dataframe(results_postcut)

# Analysis

### Parse preds

In [142]:
model = "firefunction-v2"
k = 35
sampling = "tfidf"
data_path = f"../data/dataset/reddit+shsyt_cloze/{model}/"
output_path = f"../output/reddit+shsyt_cloze"
group_paths = os.listdir(data_path)

results = {}

for group in group_paths:
    group_path = os.path.join(data_path, group)
    
    titles = []
    performers = []
    texts = []
    tags = []
    pred_tags = []

    for dataset in os.listdir(group_path):
        _texts, _tags = read_IOB_file(os.path.join(group_path, dataset, "test.IOB"))
        texts += _texts
        tags += _tags

        _output_path = os.path.join(output_path, group, model)
        for pred_file in [f for f in os.listdir(_output_path) if dataset in f]:
            _, pred_k, pred_sampling = parse_filename(pred_file)
            if pred_k == k and pred_sampling == sampling:
                pred_path = os.path.join(_output_path, pred_file)
                pred_data = read_jsonlines(pred_path)
                true_iobs, pred_iobs = get_iobs_from_data(pred_data)
                pred_tags += pred_iobs

        
        titles += [d.get("titles") for d in pred_data]
        performers += [d.get("performers") for d in pred_data]

    group_key = group.replace("_", "")
    results['_'.join((group_key, "titles"))] = titles
    results['_'.join((group_key, "performers"))] = performers
    results['_'.join((group_key, "TEXT"))] = [list(a) for a in texts]
    results['_'.join((group_key, "IOB"))] = [list(a) for a in tags]
    results['_'.join((group_key, "IOB-pred" ))] = pred_tags

_data1 = pd.read_parquet("../data/dataset/reddit+shsyt_cloze/data_template.parquet").reset_index()
_data1.columns = pd.MultiIndex.from_tuples([('', col) for col in _data1.columns])

_data2 = pd.DataFrame(results)
_data2.columns = pd.MultiIndex.from_tuples([tuple(col.split('_')) for col in _data2.columns])
     
data = pd.concat([_data1, _data2], axis=1)


In [197]:
import importlib.util
import sys
from pathlib import Path
file_path = Path("../baseline/music-ner-eacl2023/music-ner/src/ner_eval.py").resolve()
spec = importlib.util.spec_from_file_location("ner_eval", str(file_path))
ner_eval = importlib.util.module_from_spec(spec)
from ner_eval import Evaluator

def eval_pair(true_iobs, pred_iobs):
    if len(true_iobs) == len(pred_iobs):
        evaluator = Evaluator([true_iobs], [pred_iobs], ["Artist", "WoA"])
        return evaluator.evaluate()
    return None

for subset in data.columns.get_level_values(0).unique():
    if subset != '':
         data[(subset, "metrics")] = data.apply(lambda row: eval_pair(
             row[(subset, "IOB")], row[(subset, "IOB-pred")]), axis=1)


2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Imported 1 predictions for 1 true examples
2024-09-12 15:21:12 root INFO: Importe